# Installing required packages

In [ ]:
!pip install nemo_toolkit[all]
!pip install pydub
!pip install jsonlines
!wget https://sc.link/ZMv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl
import pandas as pd
import jsonlines
from sklearn.model_selection import train_test_split
from jiwer import wer, cer
import gc
import torch

from google.colab import drive
drive.mount('/content/drive')

[NeMo W 2023-05-24 21:47:33 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-24 21:47:38 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


Mounted at /content/drive


# Load the QuatrzNet model which was trained on Zapadnodvisk data

In [ ]:
asr_model_quartznet = nemo_asr.models.EncDecCTCModel.load_from_checkpoint("/content/drive/MyDrive/epoch=19-step=13880.ckpt")

[NeMo W 2023-05-23 18:53:08 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 8
    trim_silence: false
    max_duration: 37.02
    min_duration: 0.01
    num_workers: 0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-05-23 18:53:08 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a vali

[NeMo I 2023-05-23 18:53:08 features:287] PADDING: 16


# Download the manifest

In [ ]:
manifest = []
with jsonlines.open('/content/drive/MyDrive/manifest.jsonl') as f:
    for obj in f:
        manifest.append(obj)

train, test = train_test_split(manifest, test_size=0.3, random_state=22)

with jsonlines.open('train.jsonl', 'w') as writer:
    writer.write_all(train)
with jsonlines.open('test.jsonl', 'w') as writer:
    writer.write_all(test)

In [ ]:
asr_model_quartznet = asr_model_quartznet.cuda()

# Prepare function for evaluating

In [ ]:
def get_metrics(transcriptions):
    wers = []
    cers = []

    for i, transcription in enumerate(transcriptions):
        if test[i]['text'] != '' and test[i]['text'] != ' ':
          w = wer(test[i]['text'], transcription)
          wers.append(w)
          c = cer(test[i]['text'], transcription)
          cers.append(c)

    print('Mean WER: ', sum(wers)/len(wers))
    print('Mean CER: ', sum(cers)/len(cers))

In [ ]:
gc.collect()

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

if accelerator == 'gpu':
  torch.cuda.empty_cache()

# QuarztNet: transcribe

In [ ]:
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model_quartznet.transcribe(paths2audio_files=files)
get_metrics(transcriptions)

Transcribing:   0%|          | 0/170 [00:00<?, ?it/s]

Mean WER:  0.7606520648424506
Mean CER:  0.45218104547758003


In [ ]:
for i, item in enumerate(test):
    test[i]['transcription'] = transcriptions[i]
df_quartz = pd.DataFrame.from_records(test)
df_quartz.to_excel("output_quartz_opochka.xlsx") 

# Load the Conformer-Transducer model which was trained on Zapadnodvisk data

In [ ]:
asr_model_conformer = nemo_asr.models.EncDecRNNTBPEModel.restore_from('/content/drive/MyDrive/finetuned_conformer.nemo')

[NeMo I 2023-05-23 18:55:01 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-23 18:55:02 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    batch_size: 2
    shuffle: true
    num_workers: 0
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 37.02
    min_duration: 0.01
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-05-23 18:55:02 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /content/test.jsonl
    sample_rate: 16000
    

[NeMo I 2023-05-23 18:55:02 features:287] PADDING: 0


[NeMo W 2023-05-23 18:55:03 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    
[NeMo W 2023-05-23 18:55:03 rnnt:1211] `preserve_memory` was set for the Joint Model. Please be aware this will severely impact the forward-backward step time. It also might not solve OOM issues if the GPU simply does not have enough memory to compute the joint.


[NeMo I 2023-05-23 18:55:03 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-23 18:55:04 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /content/drive/MyDrive/finetuned_conformer.nemo.


# Conformer-Transducer: transcribe

In [ ]:
asr_model_conformer = asr_model_conformer.cuda()
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model_conformer.transcribe(paths2audio_files=files)
get_metrics(transcriptions[0])

Mean WER:  0.4252619251422297
Mean CER:  0.3064159860740065


In [ ]:
for i, item in enumerate(test):
    test[i]['transcription'] = transcriptions[0][i]
df_conformer = pd.DataFrame.from_records(test)
df_conformer.to_excel("output_conformer_opochka.xlsx") 

# Test on models not trained on Zapadnodvisk data

In [ ]:
asr_model_quartznet_base = nemo_asr.models.EncDecCTCModel.restore_from("/content/ZMv")
asr_model_quartznet_base = asr_model_quartznet_base.cuda()
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model_quartznet_base.transcribe(paths2audio_files=files)
get_metrics(transcriptions)

--2023-05-24 21:48:35--  https://sc.link/ZMv
Resolving sc.link (sc.link)... 37.230.233.245
Connecting to sc.link (sc.link)|37.230.233.245|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo [following]
--2023-05-24 21:48:36--  https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo
Resolving n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)... 37.18.122.129
Connecting to n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)|37.18.122.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71179174 (68M) [application/octet-stream]
Saving to: ‘ZMv’

ZMv                 100%[===================>]  67.88M  16.9MB/s    in 4.0s    

2023-05-24 21:48:42 (16.9 MB/s) - ‘ZMv’ saved [71179174/71179174]



[NeMo W 2023-05-24 21:48:43 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-05-24 21:48:43 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2023-05-24 21:48:43 features:287] PADDING: 16
[NeMo I 2023-05-24 21:48:44 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /content/ZMv.


Transcribing:   0%|          | 0/170 [00:00<?, ?it/s]

Mean WER:  0.8261613760681126
Mean CER:  0.6522794683996045


In [ ]:
asr_model_conformer_base = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_ru_conformer_transducer_large")
asr_model_conformer_base = asr_model_conformer_base.cuda()
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model_quartznet_base.transcribe(paths2audio_files=files)
get_metrics(transcriptions[0])

[NeMo I 2023-05-24 21:51:43 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_ru_conformer_transducer_large/versions/1.13.0/files/stt_ru_conformer_transducer_large.nemo to /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo
[NeMo I 2023-05-24 21:52:04 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-05-24 21:52:07 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-24 21:52:08 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-05-24 21:52:08 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: fals

[NeMo I 2023-05-24 21:52:08 features:287] PADDING: 0


[NeMo W 2023-05-24 21:52:10 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-05-24 21:52:10 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-24 21:52:11 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo.


Transcribing:   0%|          | 0/170 [00:00<?, ?it/s]

Mean WER:  1.0
Mean CER:  0.9888001809347908
